In [1]:
from dotenv import load_dotenv
import nltk
from functions.metamorphic import calculate_M_ASR
from functions.perturbations import delete_characters, add_characters, add_random_words, remplace_named_entities, replace_characters, replace_words_with_antonyms, replace_words_with_synonyms
from functions.models import request_to_bing
import pandas as pd

nltk.download('punkt')
load_dotenv()

c:\Users\tcame\AppData\Local\Programs\Python\Python39\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tcame\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import os
print(os.getenv("BING_U_COOKIE"))

134UFqBhK8c1Vq5a5E2pZIL8_aE0NBlQs-jzzmfTF2JMk84_4-VANi-v1SqEdDxBWir8Vi5MlXon-mxGMkFyUrgmYx-Tx5sGfctBvCByAiQK4WHkNgFXcp-8H-WNOVsifFfkuUUAEQlcvNstJcr44izxDbdOWn1blldgXQ2Gks0C3XVB3cmg3CYSK5aMdiuW2R5kBaapYmLIQElNxqapXHfUOsIRMqdeVsP6hwLhmGS8


In [3]:
models = [
    ('summarize', 'facebook/bart-large-cnn'),
    ('summarize', 'google/pegasus-large'),
    ('summarize', 'marianna13/flan-t5-base-summarization'),
    ('toxic', 's-nlp/roberta_toxicity_classifier'),
    ('toxic', 'citizenlab/distilbert-base-multilingual-cased-toxicity', 'inputs'),
    ('toxic', 'martin-ha/toxic-comment-model'),
    ('spam', 'rafacost/bert_base_pt_en_cased_email_spam'),
    ('spam', 'h-e-l-l-o/email-spam-classification-merged'),
    ('spam', 'dima806/email-spam-detection-roberta'),
    ('translate', 't5-base'),
    ('translate', 'allenai/wmt16-en-de-12-1'),
    ('translate', 'facebook/wmt19-en-de'),
    ('fillmask', 'bert-base-uncased', '[MASK]'),
    ('fillmask', 'vinai/bertweet-base', '<mask>'),
    ('fillmask', 'roberta-base', '<mask>'),
]


#models_to_use = [models[i] for i in [0,1,3,4,6,7,9,10,13,14]]
models_summarize = [models[0], models[1]]
models_toxic = [models[3], models[4]]
perturbations = [(delete_characters, False, 'delete_characters'), (replace_characters, False, 'replace_characters'), (add_characters, False, 'add_characters'),(replace_words_with_synonyms, False, 'replace_word_synonyms'), (replace_words_with_antonyms, True,'replace_word_antonyms'), (add_random_words, False,'add_random_words'), (remplace_named_entities, False,'remplace_named_entities')]
atribbutes = ["Robustness", "Non-determinism", "Fairness"]

In [4]:
print(await request_to_bing("What is the capital of Spain?"))

CreateConversationException: Failed to create conversation, received status: 404

In [5]:
RESULT = pd.DataFrame(columns=["Value", "Time"])

In [6]:
for m in models_summarize:
    m_type = m[0]
    for p in perturbations:
        new_a = atribbutes.copy() if m_type != "summarize" else atribbutes.copy()[:-1]
        for a in new_a:
            print(f"Model: {m[1]}, Perturbation: {p[0].__name__}, Attribute: {a}")
            result = await calculate_M_ASR(m, p[0], a, p[2],change_meaning=p[1], iterations=1)
            RESULT.loc[m[1] + " - " + p[0].__name__ + " - " + a] = (result[0], result[1])

Model: facebook/bart-large-cnn, Perturbation: delete_characters, Attribute: Robustness
Model: facebook/bart-large-cnn, Perturbation: delete_characters, Attribute: Non-determinism
PROMPT:  Delete aleatory characters from the following sentence and give me back just the result, i dont care if this change the meaning of the sentence: 
FIRST QUESTION:  A renowned New York playwright is enticed to California to write for the movies and discovers the hellish truth of Hollywood.
PROMPT:  Compare the following sentences.Say me only the % number of similarity, i dont want anyting else: 

FIRST QUESTION:  A renowned New York playwright is enticed to California to write for the movies. He discovers the hellish truth of Hollywood. The play is based on a true story about a playwright who moved to Hollywood to work on a movie. The film is directed by Tom Hanks and is set in California.
SECOND QUESTION:  Sure, here's the sentence after deleting some random characters:"renowned York playwrit is entice

In [7]:
RESULT

,Value,Time
facebook/bart-large-cnn - delete_characters - Robustness,1.0,8.264758
facebook/bart-large-cnn - delete_characters - Non-determinism,1.0,17.770170
facebook/bart-large-cnn - replace_characters - Robustness,0.0,7.813684
facebook/bart-large-cnn - replace_characters - Non-determinism,1.0,21.727678
facebook/bart-large-cnn - add_characters - Robustness,1.0,10.058627
facebook/bart-large-cnn - add_characters - Non-determinism,1.0,31.354026
facebook/bart-large-cnn - replace_words_with_synonyms - Robustness,1.0,14.693539
facebook/bart-large-cnn - replace_words_with_synonyms - Non-determinism,0.0,30.646007
facebook/bart-large-cnn - replace_words_with_antonyms - Robustness,0.0,14.946164
facebook/bart-large-cnn - replace_words_with_antonyms - Non-determinism,0.0,27.091527


In [8]:
for m in models_toxic:
    m_type = m[0]
    for p in perturbations:
        new_a = atribbutes.copy() if m_type != "summarize" else atribbutes.copy()[:-1]
        for a in new_a:
            print(f"Model: {m[1]}, Perturbation: {p[0].__name__}, Attribute: {a}")
            result = await calculate_M_ASR(m, p[0], a, p[2],change_meaning=p[1], iterations=1)
            RESULT.loc[m[1] + " - " + p[0].__name__ + " - " + a] = (result[0], result[1])

Model: s-nlp/roberta_toxicity_classifier, Perturbation: delete_characters, Attribute: Robustness
Model: s-nlp/roberta_toxicity_classifier, Perturbation: delete_characters, Attribute: Non-determinism
PROMPT:  Is the following sentence toxic? Say me only yes or no, i dont want anything else 

 Please signify the copyright information on any other images you have uploaded or will upload. Remember that images without this important information can be deleted by an administrator. If you have any questions, feel free to contact me. Thank you.  (T/C) " public domain, and fair use. Find the appropriate template in Wikipedia:Image copyright tags and place it on the image page like this: {{TemplateName}}.
Model: s-nlp/roberta_toxicity_classifier, Perturbation: delete_characters, Attribute: Fairness
PROMPT:  Is the following sentence toxic? Say me only yes or no, i dont want anything else 

FIRST QUESTION:  whats up with this stock market business? thats def. orig. research
Model: s-nlp/roberta_t

ThrottledRequestException: Request is throttled